In [ ]:
import pandas as pd
import plotly.express as px
from dash import Dash, dcc, html
from dash.dependencies import Input, Output

# Load and prepare data
df = pd.read_csv('Historical_Wildfires.csv')
df['Date'] = pd.to_datetime(df['Date'])
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month

app = Dash(__name__)

app.layout = html.Div([
    html.H1("Australian Wildfire Analysis Dashboard",
            style={'textAlign': 'center', 'color': '#2C3E50', 'padding': '20px'}),
    
    html.Div([
        html.Label('Select Region:'),
        dcc.RadioItems(
            id='region-selector',
            options=[{'label': region, 'value': region} 
                    for region in sorted(df['Region'].unique())],
            value=sorted(df['Region'].unique())[0],
            style={'margin': '10px'}
        ),
        
        html.Label('Select Year:'),
        dcc.Dropdown(
            id='year-selector',
            options=[{'label': str(year), 'value': year} 
                    for year in sorted(df['Year'].unique())],
            value=sorted(df['Year'].unique())[0],
            style={'width': '50%', 'margin': '10px'}
        )
    ], style={'padding': '20px', 'backgroundColor': '#F8F9F9'}),
    
    html.Div([
        html.Div([
            dcc.Graph(id='monthly-fire-area-pie')
        ], style={'width': '50%', 'display': 'inline-block'}),
        
        html.Div([
            dcc.Graph(id='monthly-fire-count-bar')
        ], style={'width': '50%', 'display': 'inline-block'})
    ])
])

@app.callback(
    [Output('monthly-fire-area-pie', 'figure'),
     Output('monthly-fire-count-bar', 'figure')],
    [Input('region-selector', 'value'),
     Input('year-selector', 'value')]
)
def update_charts(selected_region, selected_year):
    filtered_df = df[
        (df['Region'] == selected_region) & 
        (df['Year'] == selected_year)
    ]
    
    monthly_fire_area = filtered_df.groupby('Month')['Estimated_fire_area'].mean()
    monthly_fire_count = filtered_df.groupby('Month')['Count'].mean()
    
    pie_fig = px.pie(
        values=monthly_fire_area.values,
        names=monthly_fire_area.index,
        title=f'Monthly Average Estimated Fire Area - {selected_region} ({selected_year})'
    )
    
    bar_fig = px.bar(
        x=monthly_fire_count.index,
        y=monthly_fire_count.values,
        title=f'Monthly Average Fire Pixel Count - {selected_region} ({selected_year})'
    )
    bar_fig.update_layout(
        xaxis_title='Month',
        yaxis_title='Average Count of Pixels'
    )
    
    return pie_fig, bar_fig

if __name__ == '__main__':
    app.run_server(debug=True)